# Cluster with KMeans

## Task Description
Create a dummy dataset with 3 different labels: [a, b, c] and cluster them using KMeans. Then classify the dummy dataset using kmeans model and print out the rows of the first cluster. Finally save the kmeans model to kmeans.mlnet. Once completed, print 'task resolved' in the end.

## Task Description

Create a dummy dataset with 3 different labels: [a, b, c] and cluster them using KMeans. Then classify the dummy dataset using kmeans model and print out the rows of the first cluster. Finally save the kmeans model to kmeans.mlnet. Once completed, print 'task resolved' in the end.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

var mlContext = new MLContext(seed: 0);

// Create a list of training data points.
var dataPoints = GenerateRandomDataPoints(1000, 123);

// Convert the list of data points to an IDataView object, which is
// consumable by ML.NET API.
IDataView trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

// Define the trainer.
var pipeline = mlContext.Clustering.Trainers.KMeans(
    numberOfClusters: 3);

// Train the model.
var model = pipeline.Fit(trainingData);

// Save the model
mlContext.Model.Save(model, trainingData.Schema, "kmeans.mlnet");

// Create testing data. Use a different random seed to make it different
// from the training data.
var testData = mlContext.Data.LoadFromEnumerable(
    GenerateRandomDataPoints(500, seed: 123));

// Run the model on test data set.
var transformedTestData = model.Transform(testData);

// Convert IDataView object to a list.
var predictions = mlContext.Data.CreateEnumerable<Prediction>(
    transformedTestData, reuseRowObject: false).ToList();

// Print the rows of the first cluster.
foreach (var p in predictions.Where(x => x.PredictedLabel == 1))
    Console.WriteLine(
        $"Label: {p.Label}, Prediction: {p.PredictedLabel}");

Console.WriteLine("task resolved");

IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        int label = i % 3;
        yield return new DataPoint
        {
            Label = (uint)label,
            Features = Enumerable.Repeat(label, 50)
                .Select(index => label == 0 ? randomFloat() + 0.1f :
                    label == 1 ? randomFloat() : randomFloat() - 0.1f).ToArray()
        };
    }
}

class DataPoint
{
    [KeyType(3)]
    public uint Label { get; set; }

    [VectorType(50)]
    public float[] Features { get; set; }
}

class Prediction
{
    public uint Label { get; set; }
    public uint PredictedLabel { get; set; }
}